In [1]:
%%time
# IMPORT MODULES
import os
from pyspark.ml.feature import VectorAssembler
#from pyspark.mllib.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession

CPU times: user 351 ms, sys: 196 ms, total: 548 ms
Wall time: 547 ms


In [2]:
%%time
spark = SparkSession.builder \
        .master("local") \
        .appName("mllib_classifier") \
        .getOrCreate()

CPU times: user 20.6 ms, sys: 16.4 ms, total: 37 ms
Wall time: 5 s


In [3]:
%%time
sc = spark.sparkContext

CPU times: user 4 µs, sys: 4 µs, total: 8 µs
Wall time: 10.3 µs


In [7]:
%%time
df = spark.read.parquet("/project/ds5559/group12/processed_data/output1.parquet")

CPU times: user 2.65 ms, sys: 1.57 ms, total: 4.22 ms
Wall time: 106 ms


In [8]:
%%time
train = df.sample(.8, 124)
test = df.join(train, 'user_id', 'leftanti')
print(train.count())
print(test.count())

16362
4022
CPU times: user 2.01 ms, sys: 3.03 ms, total: 5.04 ms
Wall time: 381 ms


In [9]:
train.show(5)

+---------+--------------+--------------+---------------+------------------+----------------+
|  user_id|m2_total_spend|m1_total_spend|m1_total_events|m1_purchase_events|m1_user_sessions|
+---------+--------------+--------------+---------------+------------------+----------------+
|554090147|          0.00|      12066.53|             87|                26|               3|
|555477847|          0.00|       3110.48|             20|                 8|               1|
|537600119|          0.00|       1337.02|             24|                 7|               3|
|515125723|          0.00|       1422.27|             20|                 7|               1|
|525454715|          0.00|        909.83|             20|                 6|               4|
+---------+--------------+--------------+---------------+------------------+----------------+
only showing top 5 rows



In [21]:
%%time
assembler = VectorAssembler(inputCols = ['m1_total_spend', 'm1_total_events', 'm1_purchase_events', 'm1_user_sessions'],
                            outputCol = 'features')

tr = assembler.transform(train)
tr.select("*").show(2, truncate = False)

+---------+--------------+--------------+---------------+------------------+----------------+------------------------+
|user_id  |m2_total_spend|m1_total_spend|m1_total_events|m1_purchase_events|m1_user_sessions|features                |
+---------+--------------+--------------+---------------+------------------+----------------+------------------------+
|554090147|0.00          |12066.53      |87             |26                |3               |[12066.53,87.0,26.0,3.0]|
|555477847|0.00          |3110.48       |20             |8                 |1               |[3110.48,20.0,8.0,1.0]  |
+---------+--------------+--------------+---------------+------------------+----------------+------------------------+
only showing top 2 rows

CPU times: user 3.47 ms, sys: 3.14 ms, total: 6.61 ms
Wall time: 309 ms


In [17]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(tr)
scaledData = scalerModel.transform(tr) # Divides by the standard deviation

scaledData.select("m2_total_spend","scaledFeatures").show(2, truncate=False)

+--------------+---------------------------------------------------------------------------+
|m2_total_spend|scaledFeatures                                                             |
+--------------+---------------------------------------------------------------------------+
|0.00          |[64.22907758571034,12.890575362244295,63.87494704082937,4.708966974705326] |
|0.00          |[16.556811382294686,2.963350657987194,19.65382985871673,1.5696556582351087]|
+--------------+---------------------------------------------------------------------------+
only showing top 2 rows



In [29]:
from pyspark.sql import col
from pyspark.sql import DoubleType
train.withColumn("m2_total_spend",col("m2_total_spend").cast(DoubleType))

ImportError: cannot import name 'col' from 'pyspark.sql' (/usr/local/spark/python/pyspark/sql/__init__.py)

In [25]:
%%time
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='m2_total_spend',         # feature vector name
                      labelCol='scaledFeatures',  # target variable name
                      maxIter=10,
                      regParam=0.3, 
                      elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(scaledData)

# Print the weights and intercept for linear regression
print("Weights: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

IllegalArgumentException: requirement failed: Column m2_total_spend must be of type class org.apache.spark.ml.linalg.VectorUDT:struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually class org.apache.spark.sql.types.DecimalType:decimal(17,2).